### Setup

In [2]:
import os
import openai
import numpy as np
from tqdm.notebook import tqdm

# set up openai api key
api_key_location=os.path.expanduser(os.path.join("~/.ssh/", "openai")) 
with open(api_key_location) as f:
    openai.api_key = f.read().strip()

### Generate prompts

In [ ]:
# generate_chat_prompt will take as input a tuple of indices corresponding to concepts in `labels`
def generate_prompt(color):
    prompt = f"""What color is the following hex code? Reply with one word only.\n"""
    prompt += f'''Color 1: {color}\n'''
    return prompt

# Let's try an example
color="#000000"
prompt=generate_prompt(color)
print(prompt)
messages=[{"role":"user", "content":prompt}]
print(messages)

What color is the following hex code? Reply with one word only.
Color 1: #000000

[{'role': 'user', 'content': 'What color is the following hex code? Reply with one word only.\nColor 1: #000000\n'}]


### Configure GPT

In [2]:
temperature = 0.7 # default temperature setting
model = "gpt-3.5-turbo"
seed = 1

### Collect GPT Responses


In [3]:
def openai_chat_completion(messages, model, temperature):
    response=False
    i=0
    while not response:
        i+=1
        try:
                response = openai.ChatCompletion.create(
                    model=model,
                    messages=messages,
                    temperature=temperature,
                )
        except Exception as e:
            if i>=5: #if error persis after 5 attempts, give up
                return False, False
            if i>=4:
                print(f'Attempt {i} failed: {e}')
            elif i>=3: print(f'Attempt {i} failed.')
            time.sleep(5) # wait before pinging the API again in case error is due to rate limit
    choices = [dict(choice.items()) for choice in response.choices]
    return choices, response.created

In [4]:
# get list of colors from txt file
file = open('colors.txt', 'r')
colors = file.read().splitlines()
file.close()

In [5]:
responses = []
visited_pairs=[]

for i, color in enumerate(tqdm(colors)):
    prompt = generate_prompt(color)
    response=False
    answer=False
    attempt=0
    while not answer:
        attempt+=1
        messages=[{'role':'user', 'content':prompt}]
        choices,created=openai_chat_completion(messages, model, temperature)
        try:
            answer = choices[0]['message']['content'].strip().split()[0]
            responses.append(answer)
            
        #In case we fail to parse out an answer 10 times in a row, we skip that entry
        except Exception as e: 
            print(answer)
            print(e)
            answer=False
            if attempt>10:
                print('Error', e)

NameError: name 'tqdm' is not defined

### Analyze the data